In [1]:
import json
import os
import gzip
from nltk.tokenize import sent_tokenize
import re
import csv
from tqdm.notebook import tqdm

In [2]:
input_fn = '../data/heads/head_00.jsonl.gz'
output_fn = '../out/oxford_comma/head_train_nocommas_extract.csv'
max_context_len = 1000

In [3]:
data_train = list(gzip.open(input_fn, 'rt'))

In [4]:
fh = open(output_fn, 'wt')
csv_writer = csv.writer(fh)

# write headers
csv_writer.writerow(["line", "sentence", "contains", "index"])

30

In [5]:
regex ="([\w]{4,15}, ){2,}[\w]{4,15}(,)?\s(and|or)\s"

In [6]:
for i, line in tqdm(enumerate(data_train), total=len(data_train)):
    tokenized_line = sent_tokenize(re.sub('\n', ' ', json.loads(line)["text"]))
    
    # maximize the context
    for j in range(len(tokenized_line)-1, -1, -1):
        result = re.search(regex, tokenized_line[j])
        
        if result != None:
            index = result.span()[1]
            
            if result.group(3) == "and":
                index -= 6 
            else:
                index -= 5
                
            #This is so that our index is always at the position that is supposed to be the comma (for non oxford, this is the space)
            has_oxford = result.group(2) == ','
            if not has_oxford:
                index += 1
            
            context = ' '.join(tokenized_line[:j])
            context = context.replace(',', '')
            context = context[-max_context_len:]  
            line_with_context = context + ' ' + tokenized_line[j]
            
            csv_writer.writerow([i, line_with_context, has_oxford, len(context) + 1 + index])
        
            # to go to next pile example
            break

  0%|          | 0/20000 [00:00<?, ?it/s]

In [7]:
fh.close()

In [8]:
len(context) + index, line_with_context

(744,
 'A U.S. Army Reserve colonel is accused of punching his 12-year-old son several times in the face in a restaurant parking lot according to court documents. Terry Dione Lindon 51 of Fayetteville was charged by Fayetteville police with misdemeanor child abuse in March. He is scheduled to appear in Cumberland County District Court next week. Lindon did not return a request for comment. Lindon is in the process of retiring from the Army according to officials. He was most recently a civil affairs colonel assigned to the John F. Kennedy Special Warfare Center and School at Fort Bragg according to Army Human Resources Command. The Special Warfare Center and School, the Army’s Special Operations Center of Excellence, trains, educates, develops and manages civil affairs, psychological operations and Special Forces soldiers and leaders.')